In [1]:
import pandas as pd
import random
import numpy as  np
import math
from copy import deepcopy
from sklearn.model_selection import KFold
import operator


In [2]:
data = pd.read_csv('LoanDataset/data.csv',names = ["ID", "Age", "experience", "Annual Income","ZIPCode","Family size","Avgerage spending per month","Education Level","Mortgage Value of house","Output","securities account","certificate","internet banking","credit card"])

In [3]:
data.head()

,ID,Age,experience,Annual Income,ZIPCode,Family size,Avgerage spending per month,Education Level,Mortgage Value of house,Output,securities account,certificate,internet banking,credit card
0,1,25,1.1,49,91107,4,1.6,1.2,0,0.1,1.3,0.2,0.3,0.4
1,2701,31,5.0,39,94590,4,2.2,2.0,0,0.0,0.0,0.0,1.0,1.0
2,2716,42,18.0,54,90089,1,1.8,1.0,0,0.0,0.0,0.0,1.0,0.0
3,3359,59,35.0,40,94536,4,0.4,1.0,0,0.0,0.0,0.0,0.0,0.0
4,2492,38,14.0,80,92868,2,2.7,1.0,0,0.0,0.0,0.0,1.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 14 columns):
ID                             4500 non-null int64
Age                            4500 non-null int64
experience                     4500 non-null float64
Annual Income                  4500 non-null int64
ZIPCode                        4500 non-null int64
Family size                    4500 non-null int64
Avgerage spending per month    4500 non-null float64
Education Level                4500 non-null float64
Mortgage Value of house        4500 non-null int64
Output                         4500 non-null float64
securities account             4500 non-null float64
certificate                    4500 non-null float64
internet banking               4500 non-null float64
credit card                    4500 non-null float64
dtypes: float64(8), int64(6)
memory usage: 492.3 KB


In [5]:
data = data[1:]

In [6]:
data.drop('ID',axis=1,inplace = True)
data.drop('ZIPCode',axis=1,inplace = True)

In [7]:
data.head()

,Age,experience,Annual Income,Family size,Avgerage spending per month,Education Level,Mortgage Value of house,Output,securities account,certificate,internet banking,credit card
1,31,5.0,39,4,2.2,2.0,0,0.0,0.0,0.0,1.0,1.0
2,42,18.0,54,1,1.8,1.0,0,0.0,0.0,0.0,1.0,0.0
3,59,35.0,40,4,0.4,1.0,0,0.0,0.0,0.0,0.0,0.0
4,38,14.0,80,2,2.7,1.0,0,0.0,0.0,0.0,1.0,0.0
5,62,32.0,19,1,1.5,3.0,0,0.0,1.0,0.0,0.0,0.0


In [8]:
attributes = list(data)
data.loc[data['experience'] > 0]['experience'].mean()

20.629351535836179

In [9]:
data[data < 0] = data.loc[data['experience'] > 0]['experience'].mean()


In [10]:
data.loc[data['experience'] < 0 ]

,Age,experience,Annual Income,Family size,Avgerage spending per month,Education Level,Mortgage Value of house,Output,securities account,certificate,internet banking,credit card


In [11]:
train_data = data.sample(frac = 0.8, random_state = 200)
valid_data = data.drop(train_data.index)

In [12]:
attributes

['Age',
 'experience',
 'Annual Income',
 'Family size',
 'Avgerage spending per month',
 'Education Level',
 'Mortgage Value of house',
 'Output',
 'securities account',
 'certificate',
 'internet banking',
 'credit card']

In [13]:
numerical_attr = ['Age',
 'experience',
 'Annual Income',
 'Family size',
 'Avgerage spending per month','Mortgage Value of house']

In [14]:
categorical_attr = list(set(attributes) - set(numerical_attr))

In [15]:
categorical_attr.remove('Output')

In [16]:
categorical_attr

['certificate',
 'internet banking',
 'credit card',
 'Education Level',
 'securities account']

In [17]:
totalyncounts = dict(train_data['Output'].value_counts())
totalyncounts

{0.0: 3248, 1.0: 351}

In [18]:
totalncount = totalyncounts[0.0]
totalycount = totalyncounts[1.0]
priory = totalycount*1.0/len(train_data)
priorn = totalncount*1.0/len(train_data)
priory,priorn

(0.097527090858571822, 0.90247290914142819)

In [19]:
def zeroonecount(df):
    l = [0,0]
    x = df['Output'].value_counts()
    for i,cnt in enumerate(x.keys()):
        l[not cnt] = x[cnt]
    return l

In [20]:
def meanandsd(df,output,attr):
    tempdf = df.loc[df['Output'] == output]
    return [tempdf[attr].mean(),tempdf[attr].std()]

In [21]:
meanandsd(data,0,'Family size')

[2.3729397293972938, 1.145925614657932]

In [22]:
categorical_dict = {}
numerical_dict = {}
for category in categorical_attr:
    print(category)
    for uniq in train_data[category].unique():
        try:
            categorical_dict[category][uniq] =  zeroonecount(train_data.loc[train_data[category] == uniq])
        except:
            categorical_dict[category] = { uniq : zeroonecount(train_data.loc[train_data[category] == uniq])}

for attr in numerical_attr:
    numerical_dict[attr] = {'yes':meanandsd(train_data,1.0,attr) , 'no':meanandsd(train_data,0.0,attr)}
        

certificate
internet banking
credit card
Education Level
securities account


In [23]:
categorical_dict

{'Education Level': {1.0: [68, 1450], 2.0: [132, 870], 3.0: [151, 928]},
 'certificate': {0.0: [249, 3139], 1.0: [102, 109]},
 'credit card': {0.0: [249, 2308], 1.0: [102, 940]},
 'internet banking': {0.0: [127, 1299], 1.0: [224, 1949]},
 'securities account': {0.0: [305, 2913], 1.0: [46, 335]}}

In [24]:
numerical_dict

{'Age': {'no': [45.286022167487687, 11.429478544479084],
  'yes': [45.028490028490026, 11.562959964561113]},
 'Annual Income': {'no': [66.881773399014776, 40.761297824836497],
  'yes': [144.1082621082621, 31.877259518341194]},
 'Avgerage spending per month': {'no': [1.741514778325123, 1.576009914947988],
  'yes': [3.8571794871794869, 2.0640804993246529]},
 'Family size': {'no': [2.3691502463054186, 1.147349868152981],
  'yes': [2.6239316239316239, 1.1140133037920332]},
 'Mortgage Value of house': {'no': [52.576354679802954, 93.771726876123367],
  'yes': [91.903133903133906, 156.19903700592766]},
 'experience': {'no': [20.265129079170798, 11.232039855413042],
  'yes': [19.877492877492877, 11.546889772170834]}}

In [25]:
def guassiandist(attr,df,x,output):
    mean = numerical_dict[attr][output][0]
    std = numerical_dict[attr][output][1]
    exp = math.exp(-(math.pow(x-mean,2))/(2*math.pow(std,2)))
    ans = (1.0/math.sqrt(2*math.pi)*std)*exp
    return ans

In [26]:
guassiandist('Family size',train_data,7,'yes')

0.00019817665853273084

In [27]:
wcount = 0
rcount  = 0
tp =0
tn = 0
fp = 0
fn = 0
for rowno,row in valid_data.iterrows():
    yesprob = 1.0
    noprob = 1.0
    
    for catattr in categorical_attr:
        yesprob *= max(((1.0*categorical_dict[catattr][row[catattr]][0])/totalycount),10**-9)
        noprob *= max(((1.0*categorical_dict[catattr][row[catattr]][1])/totalncount),10**-9)
    for numattr in numerical_attr:
        yesprob *= max(guassiandist(numattr,train_data,row[numattr],'yes'),10**-9)
        noprob *= max(guassiandist(numattr,train_data,row[numattr],'no'),10**-9)
    yesprob *= priory
    noprob *= priorn
    print(yesprob,noprob)
    if yesprob >= noprob:
        decision = 1
    else:
        decision = 0
    if (row['Output'] == decision):
        rcount += 1
        if(decision == 1):
            tp += 1
        else:
            tn += 1
    else:
        wcount += 1
        if(decision == 1):
            fp += 1
        else:
            fn += 1
    print(row['Output'],decision)
print("accuracy",rcount/(rcount+wcount))
print("true positive",tp)
print("true negative",tn)
print("false positive",fp)
print("false negative",fn)
print("recall",(tp)/(tp+fn))
print("precison",(tp)/(tp+fp))
print("F1 score",2*(1/((1/((tp)/(tp+fn))) + (1/((tp)/(tp+fp))))))

2.22799949337 2.96546821387
0.0 0
0.00482349216033 25.5579564979
0.0 0
0.016359070516 9.39281280906
0.0 0
0.0543987059175 0.000286079657104
0.0 1
0.154559652087 0.00500077249064
0.0 1
1.12941591648 0.0129108149602
1.0 1
0.0280778715697 59.160613683
0.0 0
0.166471470504 114.205444604
0.0 0
0.0208643420795 107.805676171
0.0 0
0.512609021883 0.0627567673668
1.0 1
0.335840904952 33.641913015
0.0 0
0.972563573299 42.3080437677
0.0 0
0.192249742911 27.2789377023
0.0 0
0.0626768563833 85.6730647542
0.0 0
0.00282281050206 3.58254531652
0.0 0
0.179455738065 2.62288085219
0.0 0
0.00220751932395 18.0221518115
0.0 0
0.330341505409 71.2957212337
0.0 0
0.190696794324 0.0679048639936
0.0 1
0.352062644628 20.8651023226
0.0 0
0.0728702369953 0.735222029815
0.0 0
0.0337454471896 0.0097439852508
0.0 1
1.52503658347 8.74598559463
0.0 0
0.0161662582407 97.4563698457
0.0 0
0.0684514619581 3.09875296777e-05
1.0 1
0.00244333712217 14.6242272062
0.0 0
0.301245618033 94.1450134291
0.0 0
0.198238347336 23.110169

0.0 1
0.482670332437 104.920672124
0.0 0
0.374552985532 2.5641028246
0.0 0
0.147810198324 0.000456723476417
0.0 1
2.31229379843e-05 1.33634633809
0.0 0
4.56674249787e-05 0.539299453492
0.0 0
0.227481656087 72.4363720361
0.0 0
0.17302996245 2.92652923064
0.0 0
0.0734878072343 116.850456225
0.0 0
0.0022084007379 5.92482196996
0.0 0
0.104989880242 104.156428023
0.0 0
0.00510563792751 17.9074645534
0.0 0
0.00546694969905 1.18884756048
0.0 0
0.747498629497 80.7884664692
0.0 0
0.00471591776133 6.83241869353
0.0 0
1.95347694656 94.5545640533
0.0 0
0.0120774506376 3.90555944639
0.0 0
0.135013073169 17.0536527359
0.0 0
0.117346099681 49.247718117
0.0 0
0.00164866696142 36.1227103902
0.0 0
0.00893286667507 5.92523691814
0.0 0
4.11887952944 262.132716712
0.0 0
0.000144377565821 4.35521963308
0.0 0
2.65182115942 52.6940875035
0.0 0
4.97806100183 14.6248564563
0.0 0
0.397029172273 0.661517609951
0.0 0
0.0649906172133 15.0113668868
0.0 0
0.153790641425 72.7370933345
0.0 0
0.192779090686 2.6559520456

0.000268365633923 40.0994687087
0.0 0
0.000166327820378 1.00441019765
0.0 0
0.152003452486 3.59473644056
0.0 0
0.030529257114 4.49631580818
0.0 0
0.000443278784488 2.09428631707
0.0 0
0.707273043442 0.286404690693
0.0 1
0.217311059996 12.7970093115
0.0 0
0.871933328034 105.032092519
0.0 0
0.021641475479 33.9651198291
0.0 0
0.0637700061071 22.6269616476
0.0 0
0.0146183264695 72.7567807715
0.0 0
0.319836192847 0.015760977565
0.0 1
0.378527252017 0.512017177977
0.0 0
0.00152859691981 15.7134521226
0.0 0
0.0192023140056 11.008983073
0.0 0
0.583543215772 1.74108693304
0.0 0
0.00594952834225 31.7469608443
0.0 0
0.336873757197 19.0706937331
0.0 0
5.7090822616 56.543490831
0.0 0
5.66132265131 25.1539241308
0.0 0
0.0120069464313 40.7069765259
0.0 0
3.63452911951e-05 8.44598555461
0.0 0
0.00474153577853 12.8895740982
0.0 0
0.000691913673192 3.50373309691
0.0 0
17.5524816337 145.199446487
0.0 0
1.01121103239 0.021071287253
0.0 1
0.00131077028312 55.9047958642
0.0 0
0.00160090552311 51.9823954367
